In [1]:
import random
import pandas as pd
import numpy as np
import Covid19_model as CM
import matplotlib.pyplot as plt
import numpy as np
import shutil
import sys
import os.path
import math
import csv
from pyomo.environ import *
import lhsmdu
import time

# Set parameter

In [2]:
D = 1e2
n_site = 24
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 2.8e6

In [3]:
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
seed = 0
env = CM.Env_model(init_state, deter_para, in_deter_para, seed)
env.set_para_truth(para_truth)

In [5]:
tic = time.time()
random.seed(seed)
observation, state = init_state.copy(), init_state.copy()
state_buffer, new_buffer  = np.zeros([T, 6]), np.zeros([T, 4])
score, ni, ne, num_sample = 0, 0, 0, 20000
done = False
index = [0]
start_time = 6
for t in range(T):
    pset = env.para_sampling(100)
    para = pset[index[0]]
    action_buffer = [] 
    reward_buffer = []
    
    if budget > 0 and t> start_time:     
        for j in range(num_sample):  
            b0 = budget
            s0 = score
            ac = [random.random(),random.random(),random.random(),random.random(),random.random()]
            ac = [ac * n_site]
            ac = np.reshape(np.array(ac), (n_site, 5)).T
            
            action_buffer.append(ac) 
            state_ex, r, _, _, c = env.transition(state, ac, para)  
            s0 += r
            b0 -= c
            for i in range(t, T):
                if b0 > 0:
                    act = np.ones((5, n_site)) 
                    
                else:
                    act = np.zeros((5, n_site))
                next_ex, reward_ex, _, _, c = env.transition(state_ex, act, para)  
                s0 += reward_ex
                b0 -= c
                state_ex = next_ex.copy()          
            reward_buffer.append(s0)
               
        max_reward = np.argmax(np.array(reward_buffer))
        action = action_buffer[max_reward]
    else:
        action = np.zeros((5, n_site))
    next_state, obs, reward, cost, new_i, new_e, ck = env.online_state_estimate(state, observation, 
                                                                                action, pset, 100, 1)

    index = ck.argsort()[0][:1]
    score += reward
    budget -= cost
    ni += new_i
    ne += new_e
    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]), 
                  budget, score))
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]), 
                  np.sum(state[4]), np.sum(state[5]), budget, score))
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    print(action)
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time() - tic)

 Time: 0 | S: 25495832.0, E: 168.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2800000.00, reward:-343.91
 Time: 0 | S: 25495832.0, E: 168.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2800000.00, reward:-343.91
 Time: 0 | new_e:79.4, cum_e:79.4, new_i:70.9, cum_i:70.9
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
 Time: 1 | S: 25495699.7, E: 221.7, A: 0.0, I: 70.9, D: 0.0, R: 7.6, Budget: 2800000.00, reward:-831.52
 Time: 1 | S: 25495697.0, E: 214.0, A: 0.0, I: 70.9, D: 0.0, R: 0.0, Budget: 2800000.00, reward:-831.52
 Time: 1 | new_e:111.4, cum_e:190.8, new_i:98.1, cum_i:169.0
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.

 Time: 10 | S: 25474367.1, E: 435.0, A: 1766.2, I: 3008.9, D: 7.9, R: 16414.9, Budget: 2660791.65, reward:-16683.49
 Time: 10 | S: 25474537.0, E: 419.0, A: 1766.2, I: 3008.9, D: 7.9, R: 16390.0, Budget: 2660791.65, reward:-16683.49
 Time: 10 | new_e:234.1, cum_e:3733.8, new_i:391.9, cum_i:3754.6
[[0.97509629 0.97509629 0.97509629 0.97509629 0.97509629 0.97509629
  0.97509629 0.97509629 0.97509629 0.97509629 0.97509629 0.97509629
  0.97509629 0.97509629 0.97509629 0.97509629 0.97509629 0.97509629
  0.97509629 0.97509629 0.97509629 0.97509629 0.97509629 0.97509629]
 [0.94740696 0.94740696 0.94740696 0.94740696 0.94740696 0.94740696
  0.94740696 0.94740696 0.94740696 0.94740696 0.94740696 0.94740696
  0.94740696 0.94740696 0.94740696 0.94740696 0.94740696 0.94740696
  0.94740696 0.94740696 0.94740696 0.94740696 0.94740696 0.94740696]
 [0.98016104 0.98016104 0.98016104 0.98016104 0.98016104 0.98016104
  0.98016104 0.98016104 0.98016104 0.98016104 0.98016104 0.98016104
  0.98016104 0.980161

 Time: 15 | S: 25306470.7, E: 438.8, A: 1179.8, I: 3106.7, D: 12.9, R: 184791.1, Budget: 2550533.40, reward:-20765.12
 Time: 15 | S: 25306709.0, E: 422.0, A: 1179.8, I: 3106.7, D: 12.9, R: 184773.0, Budget: 2550533.40, reward:-20765.12
 Time: 15 | new_e:215.5, cum_e:4606.7, new_i:317.1, cum_i:5309.3
[[0.32607929 0.32607929 0.32607929 0.32607929 0.32607929 0.32607929
  0.32607929 0.32607929 0.32607929 0.32607929 0.32607929 0.32607929
  0.32607929 0.32607929 0.32607929 0.32607929 0.32607929 0.32607929
  0.32607929 0.32607929 0.32607929 0.32607929 0.32607929 0.32607929]
 [0.97291755 0.97291755 0.97291755 0.97291755 0.97291755 0.97291755
  0.97291755 0.97291755 0.97291755 0.97291755 0.97291755 0.97291755
  0.97291755 0.97291755 0.97291755 0.97291755 0.97291755 0.97291755
  0.97291755 0.97291755 0.97291755 0.97291755 0.97291755 0.97291755]
 [0.98936319 0.98936319 0.98936319 0.98936319 0.98936319 0.98936319
  0.98936319 0.98936319 0.98936319 0.98936319 0.98936319 0.98936319
  0.98936319 0.98

 Time: 20 | S: 25162146.3, E: 298.3, A: 1231.1, I: 3059.0, D: 17.3, R: 329248.0, Budget: 2466801.30, reward:-25597.26
 Time: 20 | S: 25162477.0, E: 278.0, A: 1231.1, I: 3059.0, D: 17.3, R: 329237.0, Budget: 2466801.30, reward:-25597.26
 Time: 20 | new_e:167.5, cum_e:5623.0, new_i:275.5, cum_i:6787.1
[[0.02688128 0.02688128 0.02688128 0.02688128 0.02688128 0.02688128
  0.02688128 0.02688128 0.02688128 0.02688128 0.02688128 0.02688128
  0.02688128 0.02688128 0.02688128 0.02688128 0.02688128 0.02688128
  0.02688128 0.02688128 0.02688128 0.02688128 0.02688128 0.02688128]
 [0.06341412 0.06341412 0.06341412 0.06341412 0.06341412 0.06341412
  0.06341412 0.06341412 0.06341412 0.06341412 0.06341412 0.06341412
  0.06341412 0.06341412 0.06341412 0.06341412 0.06341412 0.06341412
  0.06341412 0.06341412 0.06341412 0.06341412 0.06341412 0.06341412]
 [0.96174695 0.96174695 0.96174695 0.96174695 0.96174695 0.96174695
  0.96174695 0.96174695 0.96174695 0.96174695 0.96174695 0.96174695
  0.96174695 0.96

 Time: 25 | S: 25034348.3, E: 185.7, A: 985.7, I: 3031.0, D: 25.0, R: 457424.4, Budget: 2290074.51, reward:-28781.68
 Time: 25 | S: 25034717.0, E: 176.0, A: 985.7, I: 3031.0, D: 25.0, R: 457394.0, Budget: 2290074.51, reward:-28781.68
 Time: 25 | new_e:102.1, cum_e:6281.6, new_i:193.5, cum_i:7947.6
[[0.80272162 0.80272162 0.80272162 0.80272162 0.80272162 0.80272162
  0.80272162 0.80272162 0.80272162 0.80272162 0.80272162 0.80272162
  0.80272162 0.80272162 0.80272162 0.80272162 0.80272162 0.80272162
  0.80272162 0.80272162 0.80272162 0.80272162 0.80272162 0.80272162]
 [0.76565985 0.76565985 0.76565985 0.76565985 0.76565985 0.76565985
  0.76565985 0.76565985 0.76565985 0.76565985 0.76565985 0.76565985
  0.76565985 0.76565985 0.76565985 0.76565985 0.76565985 0.76565985
  0.76565985 0.76565985 0.76565985 0.76565985 0.76565985 0.76565985]
 [0.99936958 0.99936958 0.99936958 0.99936958 0.99936958 0.99936958
  0.99936958 0.99936958 0.99936958 0.99936958 0.99936958 0.99936958
  0.99936958 0.9993

...

 Time: 65 | S: 24130823.8, E: 31.8, A: 309.3, I: 1358.8, D: 72.3, R: 1363404.1, Budget: 1347313.18, reward:-60426.68
 Time: 65 | S: 24131043.0, E: 24.0, A: 309.3, I: 1358.8, D: 72.3, R: 1363481.0, Budget: 1347313.18, reward:-60426.68
 Time: 65 | new_e:16.1, cum_e:12940.5, new_i:47.2, cum_i:17616.8
[[0.32439848 0.32439848 0.32439848 0.32439848 0.32439848 0.32439848
  0.32439848 0.32439848 0.32439848 0.32439848 0.32439848 0.32439848
  0.32439848 0.32439848 0.32439848 0.32439848 0.32439848 0.32439848
  0.32439848 0.32439848 0.32439848 0.32439848 0.32439848 0.32439848]
 [0.94085235 0.94085235 0.94085235 0.94085235 0.94085235 0.94085235
  0.94085235 0.94085235 0.94085235 0.94085235 0.94085235 0.94085235
  0.94085235 0.94085235 0.94085235 0.94085235 0.94085235 0.94085235
  0.94085235 0.94085235 0.94085235 0.94085235 0.94085235 0.94085235]
 [0.99480943 0.99480943 0.99480943 0.99480943 0.99480943 0.99480943
  0.99480943 0.99480943 0.99480943 0.99480943 0.99480943 0.99480943
  0.99480943 0.9948

 Time: 70 | S: 24074334.8, E: 9.5, A: 180.8, I: 934.6, D: 73.9, R: 1420466.4, Budget: 1177365.70, reward:-60745.08
 Time: 70 | S: 24074536.0, E: 23.0, A: 180.8, I: 934.6, D: 73.9, R: 1420549.0, Budget: 1177365.70, reward:-60745.08
 Time: 70 | new_e:7.7, cum_e:12991.3, new_i:25.5, cum_i:17781.9
[[0.73052519 0.73052519 0.73052519 0.73052519 0.73052519 0.73052519
  0.73052519 0.73052519 0.73052519 0.73052519 0.73052519 0.73052519
  0.73052519 0.73052519 0.73052519 0.73052519 0.73052519 0.73052519
  0.73052519 0.73052519 0.73052519 0.73052519 0.73052519 0.73052519]
 [0.84333    0.84333    0.84333    0.84333    0.84333    0.84333
  0.84333    0.84333    0.84333    0.84333    0.84333    0.84333
  0.84333    0.84333    0.84333    0.84333    0.84333    0.84333
  0.84333    0.84333    0.84333    0.84333    0.84333    0.84333   ]
 [0.96387489 0.96387489 0.96387489 0.96387489 0.96387489 0.96387489
  0.96387489 0.96387489 0.96387489 0.96387489 0.96387489 0.96387489
  0.96387489 0.96387489 0.963874

 Time: 75 | S: 23975684.9, E: 4.4, A: 96.4, I: 577.8, D: 74.7, R: 1519561.9, Budget: 1019313.51, reward:-60881.81
 Time: 75 | S: 23975831.0, E: 22.0, A: 96.4, I: 577.8, D: 74.7, R: 1519642.0, Budget: 1019313.51, reward:-60881.81
 Time: 75 | new_e:4.0, cum_e:13013.6, new_i:13.5, cum_i:17868.3
[[0.09306046 0.09306046 0.09306046 0.09306046 0.09306046 0.09306046
  0.09306046 0.09306046 0.09306046 0.09306046 0.09306046 0.09306046
  0.09306046 0.09306046 0.09306046 0.09306046 0.09306046 0.09306046
  0.09306046 0.09306046 0.09306046 0.09306046 0.09306046 0.09306046]
 [0.82827243 0.82827243 0.82827243 0.82827243 0.82827243 0.82827243
  0.82827243 0.82827243 0.82827243 0.82827243 0.82827243 0.82827243
  0.82827243 0.82827243 0.82827243 0.82827243 0.82827243 0.82827243
  0.82827243 0.82827243 0.82827243 0.82827243 0.82827243 0.82827243]
 [0.96908891 0.96908891 0.96908891 0.96908891 0.96908891 0.96908891
  0.96908891 0.96908891 0.96908891 0.96908891 0.96908891 0.96908891
  0.96908891 0.96908891 0

 Time: 80 | S: 23913371.1, E: 1.2, A: 50.0, I: 356.6, D: 75.2, R: 1582145.9, Budget: 826003.35, reward:-60941.12
 Time: 80 | S: 23913481.0, E: 0.0, A: 50.0, I: 356.6, D: 75.2, R: 1582226.0, Budget: 826003.35, reward:-60941.12
 Time: 80 | new_e:0.7, cum_e:13020.9, new_i:6.0, cum_i:17910.2
[[0.33679573 0.33679573 0.33679573 0.33679573 0.33679573 0.33679573
  0.33679573 0.33679573 0.33679573 0.33679573 0.33679573 0.33679573
  0.33679573 0.33679573 0.33679573 0.33679573 0.33679573 0.33679573
  0.33679573 0.33679573 0.33679573 0.33679573 0.33679573 0.33679573]
 [0.91600799 0.91600799 0.91600799 0.91600799 0.91600799 0.91600799
  0.91600799 0.91600799 0.91600799 0.91600799 0.91600799 0.91600799
  0.91600799 0.91600799 0.91600799 0.91600799 0.91600799 0.91600799
  0.91600799 0.91600799 0.91600799 0.91600799 0.91600799 0.91600799]
 [0.99859595 0.99859595 0.99859595 0.99859595 0.99859595 0.99859595
  0.99859595 0.99859595 0.99859595 0.99859595 0.99859595 0.99859595
  0.99859595 0.99859595 0.998

 Time: 85 | S: 23730540.3, E: 3.6, A: 23.6, I: 216.6, D: 75.5, R: 1765140.3, Budget: 714617.17, reward:-60990.74
 Time: 85 | S: 23730661.0, E: 0.0, A: 23.6, I: 216.6, D: 75.5, R: 1765219.0, Budget: 714617.17, reward:-60990.74
 Time: 85 | new_e:1.9, cum_e:13028.3, new_i:4.2, cum_i:17933.6
[[0.78570785 0.78570785 0.78570785 0.78570785 0.78570785 0.78570785
  0.78570785 0.78570785 0.78570785 0.78570785 0.78570785 0.78570785
  0.78570785 0.78570785 0.78570785 0.78570785 0.78570785 0.78570785
  0.78570785 0.78570785 0.78570785 0.78570785 0.78570785 0.78570785]
 [0.953137   0.953137   0.953137   0.953137   0.953137   0.953137
  0.953137   0.953137   0.953137   0.953137   0.953137   0.953137
  0.953137   0.953137   0.953137   0.953137   0.953137   0.953137
  0.953137   0.953137   0.953137   0.953137   0.953137   0.953137  ]
 [0.99866364 0.99866364 0.99866364 0.99866364 0.99866364 0.99866364
  0.99866364 0.99866364 0.99866364 0.99866364 0.99866364 0.99866364
  0.99866364 0.99866364 0.99866364 

In [6]:
(time.time() - tic) / ( T - start_time + 1)

89.97464247310863